# BEC-Based Inertial Sensor Simulation

This notebook demonstrates the simulation of a Bose-Einstein Condensate (BEC) based inertial sensor. We'll explore:

1. BEC State Initialization
2. Trap and Interaction Configuration
3. Ground State Preparation
4. Acceleration Sensing
5. Analysis and Visualization

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import sys
sys.path.append('..')

from src.core import BECState
from src.physics import AccelerationSensor, TrapPotential
from src.utils.numerical_methods import GPESolver
from src.visualization import BECVisualizer, PhaseVisualizer

: 

## 1. Initialize BEC System

First, we'll create our BEC state with initial parameters.

In [ ]:
# System parameters
params = {
    'grid_points': 256,
    'box_size': 20.0,  # in trap oscillator units
    'trap_frequency': 100.0,  # Hz
    'interaction_strength': 1.0,  # dimensionless g
    'atom_number': 1e5
}

# Create BEC state
bec = BECState(**params)
print(f"System initialized with {params['atom_number']} atoms")

# Visualize initial state
viz = BECVisualizer(bec)
fig, ax = viz.plot_density(title="Initial BEC Density Profile")
plt.show()

## 2. Configure Trap and Find Ground State

Now we'll set up the trapping potential and find the ground state using imaginary time evolution.

In [ ]:
# Create trap potential
trap = TrapPotential(
    frequency=params['trap_frequency'],
    center=(0.0, 0.0, 0.0),
    anisotropy=(1.0, 1.0, 1.0)
)

# Initialize GPE solver
solver = GPESolver(
    Nx=params['grid_points'],
    dx=bec._grid[1] - bec._grid[0],
    dt=1e-3,
    g=params['interaction_strength']
)

# Find ground state
V = trap.harmonic_potential(bec._grid)
ground_state, ground_energy = solver.imaginary_time(
    psi_initial=bec.wavefunction,
    V=V,
    max_iter=1000,
    tolerance=1e-10
)

# Update BEC state
bec.wavefunction = ground_state

# Visualize ground state
fig, ax = viz.plot_density(title="Ground State Density Profile")
plt.show()

print(f"Ground state energy: {ground_energy:.2f} (in trap units)")

## 3. Acceleration Sensing Setup

We'll now configure and initialize the acceleration sensor.

In [ ]:
# Create acceleration sensor
sensor = AccelerationSensor(
    bec_state=bec,
    sensitivity=1e-6,  # m/s²
    measurement_time=0.1,  # seconds
    noise_level=1e-3
)

# Calculate theoretical sensitivity limit
sensitivity_limit = sensor.get_sensitivity_limit()
print(f"Theoretical sensitivity limit: {sensitivity_limit:.2e} m/s²")

## 4. Acceleration Measurement

Let's simulate an acceleration measurement with a known test acceleration.

In [ ]:
# Apply test acceleration
test_acceleration = 1e-5  # m/s²

# Perform measurement
measured_acc, uncertainty = sensor.measure_acceleration()

print(f"Applied acceleration: {test_acceleration:.2e} m/s²")
print(f"Measured acceleration: {measured_acc:.2e} ± {uncertainty:.2e} m/s²")

# Plot measurement results
phase_viz = PhaseVisualizer()
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 10))

# Phase evolution
times = np.array(sensor.history['time']) * 1e3  # Convert to ms
phase_gradients = np.array(sensor.history['phase_gradient'])

ax1.plot(times, phase_gradients, 'b.-', label='Phase Gradient')
ax1.set_xlabel('Time (ms)')
ax1.set_ylabel('Phase Gradient')
ax1.grid(True)
ax1.legend()

# Center of mass motion
com_positions = np.array(sensor.history['com_position'])
ax2.plot(times, com_positions, 'r.-', label='COM Position')
ax2.set_xlabel('Time (ms)')
ax2.set_ylabel('Position (a.u.)')
ax2.grid(True)
ax2.legend()

plt.tight_layout()
plt.show()

## 5. Sensitivity Analysis

Finally, let's analyze the sensor's response over a range of accelerations.

In [ ]:
# Test range of accelerations
accelerations = np.logspace(-7, -4, 10)  # m/s²
measurements = []
uncertainties = []

for acc in accelerations:
    # Reset BEC to ground state
    bec.wavefunction = ground_state.copy()
    
    # Measure acceleration
    measured, uncertainty = sensor.measure_acceleration()
    measurements.append(measured)
    uncertainties.append(uncertainty)

# Plot results
measurements = np.array(measurements)
uncertainties = np.array(uncertainties)

plt.figure(figsize=(10, 6))
plt.errorbar(accelerations, measurements, yerr=uncertainties,
             fmt='bo-', capsize=5, label='Measurements')
plt.plot(accelerations, accelerations, 'k--', label='Ideal Response')

plt.xscale('log')
plt.yscale('log')
plt.xlabel('Applied Acceleration (m/s²)')
plt.ylabel('Measured Acceleration (m/s²)')
plt.title('Sensor Response Characterization')
plt.grid(True)
plt.legend()
plt.show()

# Calculate and print performance metrics
relative_error = np.abs(measurements - accelerations) / accelerations
print(f"Average relative error: {np.mean(relative_error):.2%}")
print(f"Minimum detectable acceleration: {np.min(accelerations[measurements > uncertainties]):.2e} m/s²")